# LOTUS demo

## APIs

In [3]:
import bigframes.pandas as bpd
from bigframes.ml.llm import GeminiTextGenerator

data = {
    "Course Name": [
        "Probability and Random Processes",
        "Optimization Methods in Engineering",
        "Digital Design and Integrated Circuits",
        "Computer Security",
        "Operating Systems and Systems Programming",
        "Compilers",
        "Computer Networks",
        "Deep Learning",
        "Graphics",
        "Databases",
        "Art History",
    ]
}
df = bpd.DataFrame(data)
 
model = GeminiTextGenerator(model_name="gemini-1.5-flash-001")

### 1. `sem_filter`

In [4]:
predict_df = df.sem_filter("{Course Name} requires a lot of math", model)
predict_df

/usr/local/google/home/sycai/src/python-bigquery-dataframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,Course Name,primary_results,primary_scores
0,Probability and Random Processes,True,0.95
1,Optimization Methods in Engineering,True,0.95
2,Digital Design and Integrated Circuits,True,0.95
3,Computer Security,False,0.9
4,Operating Systems and Systems Programming,False,0.8
5,Compilers,True,0.8
6,Computer Networks,False,0.75
7,Deep Learning,True,0.95
8,Graphics,True,0.85
9,Databases,False,0.95


### 2. `sem_join`

In [1]:
import bigframes.pandas as bpd
from bigframes.ml.llm import GeminiTextGenerator

data = {
    "Course Name": [
        "Probability and Random Processes",
        "Optimization Methods in Engineering",
        "Digital Design and Integrated Circuits",
        "Computer Security",
        "Operating Systems and Systems Programming",
        "Compilers",
        "Computer Networks",
        "Deep Learning",
        "Graphics",
        "Databases",
        "Art History",
    ]
}
df = bpd.DataFrame(data)
 
model = GeminiTextGenerator(model_name="gemini-1.5-flash-001")

# predict_df = df.sem_filter("Taking {Course Name} will make me better at {Skill}", model)
# predict_df
skills_df = bpd.DataFrame({"Skill": ["Art", "Cryptography", "Baking"]})

join_df = df.sem_join(skills_df, "Taking {Course Name} will make me better at {Skill}", primary_model=model)
join_df

/usr/local/google/home/chelsealin/src/bigframes/bigframes/core/global_session.py:113: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return func(get_global_session(), *args, **kwargs)


/usr/local/google/home/chelsealin/src/bigframes/bigframes/core/__init__.py:92: PreviewWarning: Interpreting JSON column(s) as StringDtype. This behavior may change in future versions.
  warnings.warn(


,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,False 0.99,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
1,False 0.95,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
2,False 0.99,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
3,False 0.99,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
4,False 0.85,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
...,...,...,...,...
28,False 0.80,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
29,False 0.99,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
30,True 0.80,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
31,False 0.99,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...


In [4]:
join_df[join_df["ml_generate_text_llm_result"].str.lower().str.contains("true")]

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
10,True 0.85,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
19,True 0.75,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
24,True 0.60,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...
30,True 0.80,"[{""category"":1,""probability"":1,""probability_sc...",,System Instruction: The user will povide a cla...


## Optimizations

### Cascade Models

In [2]:
# TODO